In [1]:
%load_ext lab_black

In [2]:
import pandas as pd

In [3]:
import duckdb
import pandas as pd
import sqlalchemy

In [4]:
import os

os.chdir("../")

In [5]:
os.getcwd()

'/home/antonio/git/trending-github'

In [6]:
%load_ext sql

In [7]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [8]:
%sql duckdb:///repos.duckdb

In [9]:
%%sql 
create table if not exists repos (
      insert_date date,
      full_name varchar,
      html_url varchar,
      url varchar,
      language varchar,
      homepage varchar,
      description varchar,
      forks_count float,
      stargazers_count float,
      topics varchar[],
      primary key(insert_date, full_name)
    )

""


In [10]:
%sql select count(*) from repos

,count_star()
0,8306


# Top Repos

In [11]:
%%sql
select full_name,
       html_url,
       homepage,
       insert_date,
       stargazers_count,
       forks_count,
       topics
from repos
where insert_date = (select max(insert_date) from repos)
-- and stargazers_count < 5000
order by stargazers_count desc
limit 30

,full_name,html_url,homepage,insert_date,stargazers_count,forks_count,topics
0,public-apis/public-apis,https://github.com/public-apis/public-apis,http://public-apis.org,2023-01-13,224512.0,25581.0,"[api, apis, dataset, development, free, list, ..."
1,donnemartin/system-design-primer,https://github.com/donnemartin/system-design-p...,,2023-01-13,208388.0,37330.0,"[design, design-patterns, design-system, devel..."
2,vinta/awesome-python,https://github.com/vinta/awesome-python,https://awesome-python.com/,2023-01-13,153188.0,22431.0,"[awesome, collections, python, python-framewor..."
3,TheAlgorithms/Python,https://github.com/TheAlgorithms/Python,https://the-algorithms.com/,2023-01-13,150930.0,38697.0,"[algorithm, algorithm-competitions, algorithms..."
4,jackfrued/Python-100-Days,https://github.com/jackfrued/Python-100-Days,,2023-01-13,129504.0,47895.0,[]
5,ytdl-org/youtube-dl,https://github.com/ytdl-org/youtube-dl,http://ytdl-org.github.io/youtube-dl/,2023-01-13,116524.0,8519.0,[]
6,huggingface/transformers,https://github.com/huggingface/transformers,https://huggingface.co/transformers,2023-01-13,77950.0,17538.0,"[bert, deep-learning, flax, hacktoberfest, jax..."
7,nvbn/thefuck,https://github.com/nvbn/thefuck,,2023-01-13,75156.0,3307.0,"[python, shell]"
8,tensorflow/models,https://github.com/tensorflow/models,,2023-01-13,74990.0,46133.0,[]
9,django/django,https://github.com/django/django,https://www.djangoproject.com/,2023-01-13,68151.0,28435.0,"[apps, django, framework, models, orm, python,..."


# Top 10 topics

In [14]:
%%sql
with t as (
select full_name,
       html_url,
       homepage,
       insert_date,
       stargazers_count,
       forks_count,
       unnest(topics) as topic
from repos
where insert_date = (select max(insert_date) from repos)
)
select topic, count(distinct full_name) num_repos
from t
group by topic
order by count(distinct full_name) desc
limit 15

,topic,num_repos
0,python,869
1,deep-learning,280
2,machine-learning,256
3,pytorch,233
4,hacktoberfest,148
5,tensorflow,138
6,python3,111
7,data-science,89
8,computer-vision,78
9,nlp,69


# top 15 repos per topic

In [16]:
%%sql
with t as (
select full_name,
       html_url,
       homepage,
       insert_date,
       stargazers_count,
       forks_count,
       unnest(topics) as topic
from repos
where insert_date = (select max(insert_date) from repos)
)

,top10_topic as (
select topic, count(distinct full_name) num_repos
from t
group by topic
order by count(distinct full_name) desc
limit 10
)
,top10_join as (
    select t.*
    from t join top10_topic on t.topic = top10_topic.topic
)
,final as (
select *, row_number() over(partition by topic order by stargazers_count desc) as rn
from top10_join)

select *
from final
where rn <= 15

,full_name,html_url,homepage,insert_date,stargazers_count,forks_count,topic,rn
0,d2l-ai/d2l-zh,https://github.com/d2l-ai/d2l-zh,http://zh.d2l.ai,2023-01-13,37217.0,8337.0,computer-vision,1
1,junyanz/pytorch-CycleGAN-and-pix2pix,https://github.com/junyanz/pytorch-CycleGAN-an...,,2023-01-13,19030.0,5635.0,computer-vision,2
2,d2l-ai/d2l-en,https://github.com/d2l-ai/d2l-en,https://D2L.ai,2023-01-13,16117.0,3494.0,computer-vision,3
3,huggingface/datasets,https://github.com/huggingface/datasets,https://huggingface.co/docs/datasets,2023-01-13,15027.0,1966.0,computer-vision,4
4,pytorch/vision,https://github.com/pytorch/vision,https://pytorch.org/vision,2023-01-13,13102.0,6454.0,computer-vision,5
...,...,...,...,...,...,...,...,...
145,onnx/onnx,https://github.com/onnx/onnx,https://onnx.ai/,2023-01-13,13933.0,3262.0,tensorflow,11
146,horovod/horovod,https://github.com/horovod/horovod,http://horovod.ai,2023-01-13,12950.0,2147.0,tensorflow,12
147,davidsandberg/facenet,https://github.com/davidsandberg/facenet,,2023-01-13,12841.0,4780.0,tensorflow,13
148,microsoft/nni,https://github.com/microsoft/nni,https://nni.readthedocs.io,2023-01-13,12394.0,1736.0,tensorflow,14


In [12]:
%%sql
select full_name,
       html_url,
       homepage,
       insert_date,
       stargazers_count,
       forks_count
       -- unnest(topics) as topic
from repos
where full_name like '%instrument%'

,full_name,html_url,homepage,insert_date,stargazers_count,forks_count
0,joerick/pyinstrument,https://github.com/joerick/pyinstrument,https://pyinstrument.readthedocs.io/,2023-01-07,5008.0,218.0
1,joerick/pyinstrument,https://github.com/joerick/pyinstrument,https://pyinstrument.readthedocs.io/,2023-01-08,5017.0,218.0
2,joerick/pyinstrument,https://github.com/joerick/pyinstrument,https://pyinstrument.readthedocs.io/,2023-01-13,5032.0,219.0


# Trend

In [9]:
26093 / 25409

1.026919595418946

In [10]:
%%sql
df << with final as (
select full_name,
       html_url,
       homepage,
       topics,
       insert_date,
    stargazers_count,   
    stargazers_count-lag(stargazers_count) over(partition by full_name order by insert_date) as stars,
    lag(insert_date) over(partition by full_name order by insert_date) as prev_date

from repos
)
select full_name,
       html_url,
       homepage,
       topics,
    stargazers_count,
       cast(insert_date as date) as insert_date,
       coalesce(stars/ date_diff('day', cast(prev_date as date) ,cast(insert_date as date)),-99999) as days
    
from final
where stars is not null
order by coalesce(stars/ date_diff('day', cast(prev_date as date) ,cast(insert_date as date)),-99999) desc
--    (stargazers_count/cast(prev as float))-1 perc
--from final
--where prev is not null
--order by (stargazers_count/cast(prev as float))-1 desc

Returning data to local variable df


In [16]:
df.head(30)

,full_name,html_url,topics,insert_date,star_diff,date_dff,delta,l,avg
1965,openai/openai-cookbook,https://github.com/openai/openai-cookbook,[],"[2023-01-08, 2023-01-07]","[3346.0, 603.0]","[5, 1]","[669.2000122070312, 603.0]",2,1211.900012
1306,karpathy/minGPT,https://github.com/karpathy/minGPT,[],"[2023-01-08, 2023-01-07]","[745.0, 465.0]","[5, 1]","[149.0, 465.0]",2,567.500000
2382,public-apis/public-apis,https://github.com/public-apis/public-apis,"[api, apis, dataset, development, free, list, ...","[2023-01-08, 2023-01-07]","[1075.0, 188.0]","[5, 1]","[215.0, 188.0]",2,384.200000
936,donnemartin/system-design-primer,https://github.com/donnemartin/system-design-p...,"[design, design-patterns, design-system, devel...","[2023-01-08, 2023-01-07]","[620.0, 223.0]","[5, 1]","[124.0, 223.0]",2,324.700000
24,AUTOMATIC1111/stable-diffusion-webui,https://github.com/AUTOMATIC1111/stable-diffus...,[],"[2023-01-08, 2023-01-07]","[485.0, 129.0]","[5, 1]","[97.0, 129.0]",2,213.100000
2301,vinta/awesome-python,https://github.com/vinta/awesome-python,"[awesome, collections, python, python-framewor...","[2023-01-08, 2023-01-07]","[573.0, 106.0]","[5, 1]","[114.5999984741211, 106.0]",2,209.999998
1907,mouredev/Hello-Python,https://github.com/mouredev/Hello-Python,"[fastapi, mongodb, python, python3, tutorial]","[2023-01-08, 2023-01-07]","[543.0, 80.0]","[5, 1]","[108.5999984741211, 80.0]",2,180.599998
427,Sanster/lama-cleaner,https://github.com/Sanster/lama-cleaner,"[inpainting, lama, latent-diffusion, mat, pyto...","[2023-01-08, 2023-01-07]","[685.0, 5.0]","[5, 1]","[137.0, 5.0]",2,141.500000
1904,microsoft/unilm,https://github.com/microsoft/unilm,"[beit, beit-3, deepnet, document-ai, document-...","[2023-01-08, 2023-01-07]","[506.0, 37.0]","[5, 1]","[101.19999694824219, 37.0]",2,134.499997
541,Z4nzu/hackingtool,https://github.com/Z4nzu/hackingtool,"[allinonehackingtool, besthackingtool, ctf-too...","[2023-01-08, 2023-01-07]","[394.0, 56.0]","[5, 1]","[78.80000305175781, 56.0]",2,129.200003


In [12]:
%%sql
df << with 
final_tmp as (
select full_name, html_url, homepage, topics,
    insert_date,
    lag(insert_date) over(partition by full_name order by insert_date desc) as prev_date,
    -stargazers_count + lag(stargazers_count) over(partition by full_name order by insert_date desc) as star_diff
from repos
)
,final as (
    select *, -date_diff('day', cast(prev_date as date) ,cast(insert_date as date)) as date_dff
    from final_tmp
    where date_dff is not null
)
select full_name, html_url,  topics, array_agg(insert_date) as insert_date, array_agg(star_diff) star_diff, array_agg(date_dff) as date_dff,
    array_agg(star_diff/greatest(date_dff,1)) as delta 
from final
group by full_name, html_url, topics

Returning data to local variable df


In [13]:
df["l"] = df.delta.apply(len)
df.query("l==3")

,full_name,html_url,topics,insert_date,star_diff,date_dff,delta,l
256,LonamiWebs/Telethon,https://github.com/LonamiWebs/Telethon,"[hacktoberfest, library, mtproto, python-libra...","[2023-01-08, 2023-01-07, 2023-01-07]","[13.0, 1.0, 0.0]","[5, 1, 0]","[2.5999999046325684, 1.0, 0.0]",3
258,Lyken17/pytorch-OpCounter,https://github.com/Lyken17/pytorch-OpCounter,[],"[2023-01-08, 2023-01-08, 2023-01-07]","[7.0, 0.0, 2.0]","[5, 0, 1]","[1.399999976158142, 0.0, 2.0]",3
667,aws/aws-cli,https://github.com/aws/aws-cli,"[aws, aws-cli, cloud, cloud-management]","[2023-01-08, 2023-01-08, 2023-01-07]","[19.0, 0.0, 2.0]","[5, 0, 1]","[3.799999952316284, 0.0, 2.0]",3
752,canonical/microk8s,https://github.com/canonical/microk8s,"[cicd, developer-workstations, hacktoberfest, ...","[2023-01-08, 2023-01-08, 2023-01-07]","[16.0, 0.0, -1.0]","[5, 0, 1]","[3.200000047683716, 0.0, -1.0]",3
1388,madhavanmalolan/awesome-reactnative-ui,https://github.com/madhavanmalolan/awesome-rea...,[],"[2023-01-08, 2023-01-07, 2023-01-07]","[0.0, 1.0, 0.0]","[5, 1, 0]","[0.0, 1.0, 0.0]",3
1409,matplotlib/cheatsheets,https://github.com/matplotlib/cheatsheets,"[cheatsheet, matplotlib, python]","[2023-01-08, 2023-01-08, 2023-01-07]","[6.0, 0.0, 1.0]","[5, 0, 1]","[1.2000000476837158, 0.0, 1.0]",3
1549,orakaro/rainbowstream,https://github.com/orakaro/rainbowstream,"[python, rainbowstream, terminal, twitter]","[2023-01-08, 2023-01-08, 2023-01-07]","[4.0, 0.0, -1.0]","[5, 0, 1]","[0.800000011920929, 0.0, -1.0]",3
1567,iGhibli/iOS-DeviceSupport,https://github.com/iGhibli/iOS-DeviceSupport,[],"[2023-01-08, 2023-01-08, 2023-01-07]","[11.0, 0.0, 3.0]","[5, 0, 1]","[2.200000047683716, 0.0, 3.0]",3
1726,huggingface/accelerate,https://github.com/huggingface/accelerate,[],"[2023-01-13, 2023-01-08, 2023-01-07]","[0.0, 25.0, 3.0]","[0, 5, 1]","[0.0, 5.0, 3.0]",3
1819,matplotlib/matplotlib,https://github.com/matplotlib/matplotlib,"[data-science, data-visualization, gtk, hackto...","[2023-01-13, 2023-01-08, 2023-01-07]","[0.0, 38.0, 5.0]","[0, 5, 1]","[0.0, 7.599999904632568, 5.0]",3


In [17]:
import numpy as np
from functools import partial


def smoothing_avg(values: list[float], coeff: float, max_length: int = None) -> float:
    size = len(values) if max_length is None else min(max_length, len(values))
    coefficients = np.power(coeff, np.arange(size))
    return (np.array(values).dot(coefficients).item()) / coefficients.sum().item()


smoothing_avg_partial = partial(smoothing_avg, coeff=0.9)
df = df.eval("avg = delta.apply(@smoothing_avg_partial)")
df = df.sort_values("avg", ascending=False)
df

,full_name,html_url,topics,insert_date,star_diff,date_dff,delta,l,avg
1965,openai/openai-cookbook,https://github.com/openai/openai-cookbook,[],"[2023-01-08, 2023-01-07]","[3346.0, 603.0]","[5, 1]","[669.2000122070312, 603.0]",2,637.842112
1306,karpathy/minGPT,https://github.com/karpathy/minGPT,[],"[2023-01-08, 2023-01-07]","[745.0, 465.0]","[5, 1]","[149.0, 465.0]",2,298.684211
2382,public-apis/public-apis,https://github.com/public-apis/public-apis,"[api, apis, dataset, development, free, list, ...","[2023-01-08, 2023-01-07]","[1075.0, 188.0]","[5, 1]","[215.0, 188.0]",2,202.210526
936,donnemartin/system-design-primer,https://github.com/donnemartin/system-design-p...,"[design, design-patterns, design-system, devel...","[2023-01-08, 2023-01-07]","[620.0, 223.0]","[5, 1]","[124.0, 223.0]",2,170.894737
24,AUTOMATIC1111/stable-diffusion-webui,https://github.com/AUTOMATIC1111/stable-diffus...,[],"[2023-01-08, 2023-01-07]","[485.0, 129.0]","[5, 1]","[97.0, 129.0]",2,112.157895
...,...,...,...,...,...,...,...,...,...
2468,tianzhi0549/FCOS,https://github.com/tianzhi0549/FCOS,"[anchor-free, computer-vision, fcos, iccv2019,...",[2023-01-07],[-1.0],[1],[-1.0],1,-1.000000
2089,pypa/setuptools,https://github.com/pypa/setuptools,[],[2023-01-07],[-1.0],[1],[-1.0],1,-1.000000
1621,alexjc/neural-enhance,https://github.com/alexjc/neural-enhance,[],"[2023-01-08, 2023-01-07]","[-2.0, -2.0]","[5, 1]","[-0.4000000059604645, -2.0]",2,-1.157895
608,anasty17/mirror-leech-telegram-bot,https://github.com/anasty17/mirror-leech-teleg...,"[aria2c, bot, clone, download, download-videos...",[2023-01-07],[-2.0],[1],[-2.0],1,-2.000000


1965    1211.900012
2382     384.200000
1306     567.500000
427      141.500000
936      324.700000
           ...     
1630      -0.800000
2527      -0.800000
2468      -1.000000
2089      -1.000000
608       -2.000000
Name: delta, Length: 2764, dtype: float64

In [27]:
np.arange(10)